In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [2]:
def preprocess_image(image):
    image = tf.image.resize(image, image_size)  # Resize image
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

In [3]:
def load_dataset(folder_path, batch_size, image_size=(64, 64)):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        folder_path,
        label_mode=None,
        image_size=image_size,
        batch_size=batch_size
    )
    dataset = dataset.map(preprocess_image)
    return dataset

In [14]:
image_size = (64, 64)
batch_size = 32
latent_dim = 100
epochs = 20

In [6]:
image_folder = 'images'  
dataset = load_dataset(image_folder, batch_size)
print(f"Dataset loaded with batch size {batch_size} and image size {image_size}")

Found 63565 files belonging to 1 classes.
Dataset loaded with batch size 32 and image size (64, 64)


In [7]:
def build_generator(latent_dim):
    model = Sequential([
        Dense(8*8*256, input_dim=latent_dim),
        Reshape((8, 8, 256)),
        BatchNormalization(),
        LeakyReLU(0.2),
        
        Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),

        Conv2DTranspose(64, (4,4), strides=(2,2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),

        Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', activation='tanh')
    ])
    return model


In [ ]:

def build_discriminator(input_shape=(64, 64, 3)):
    model = Sequential([
        Conv2D(64, (4,4), strides=(2,2), padding='same', input_shape=input_shape),
        LeakyReLU(0.2),

        Conv2D(128, (4,4), strides=(2,2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),

        Conv2D(256, (4,4), strides=(2,2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),

        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

In [9]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    return model

In [10]:
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)


In [11]:
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [27]:
import tensorflow as tf
import numpy as np

# Calculate total images and batches per epoch
total_images = tf.data.experimental.cardinality(dataset).numpy() * batch_size
total_steps_per_epoch = total_images // batch_size

# Training function with structured print statements for each epoch and step
def train_gan(gan, generator, discriminator, dataset, latent_dim, epochs=1000, batch_size=32, log_interval=100):
    d_loss_real, d_loss_fake, g_loss = [], [], []
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        d_loss_real_epoch, d_loss_fake_epoch, g_loss_epoch = [], [], []
        step = 0
        
        for real_images in dataset:
            step += 1
            half_batch = batch_size // 2

            # Train Discriminator on Real Data
            real_labels = tf.ones((real_images.shape[0], 1))
            d_loss_real_batch = discriminator.train_on_batch(real_images, real_labels)
            if isinstance(d_loss_real_batch, list):  # if it returns a list, take the first element
                d_loss_real_batch = d_loss_real_batch[0]
            d_loss_real_epoch.append(d_loss_real_batch)

            # Train Discriminator on Fake Data
            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_images = generator.predict(noise, verbose=0)
            fake_labels = tf.zeros((half_batch, 1))
            d_loss_fake_batch = discriminator.train_on_batch(fake_images, fake_labels)
            if isinstance(d_loss_fake_batch, list):  # if it returns a list, take the first element
                d_loss_fake_batch = d_loss_fake_batch[0]
            d_loss_fake_epoch.append(d_loss_fake_batch)

            # Train Generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            real_labels_for_gen = tf.ones((batch_size, 1))
            g_loss_batch = gan.train_on_batch(noise, real_labels_for_gen)
            if isinstance(g_loss_batch, list):  # if it returns a list, take the first element
                g_loss_batch = g_loss_batch[0]
            g_loss_epoch.append(g_loss_batch)

            # Print progress for each step
            print(f"Step {step}/{total_steps_per_epoch} | "
                  f"D Loss Real: {d_loss_real_batch:.4f}, "
                  f"D Loss Fake: {d_loss_fake_batch:.4f}, "
                  f"G Loss: {g_loss_batch:.4f}", end="\r")

        # Append average losses for each epoch
        d_loss_real.append(np.mean(d_loss_real_epoch))
        d_loss_fake.append(np.mean(d_loss_fake_epoch))
        g_loss.append(np.mean(g_loss_epoch))

        # Summary for each epoch
        print(f"\nEpoch {epoch+1} Summary - D Loss Real: {d_loss_real[-1]:.4f}, "
              f"D Loss Fake: {d_loss_fake[-1]:.4f}, G Loss: {g_loss[-1]:.4f}")
    
    return d_loss_real, d_loss_fake, g_loss



In [28]:

# Run the updated training function
d_loss_real, d_loss_fake, g_loss = train_gan(gan, generator, discriminator, dataset, latent_dim, epochs=epochs)


Epoch 1/20
Step 1987/1987 | D Loss Real: 0.7524, D Loss Fake: 7.1664, G Loss: 0.0008
Epoch 1 Summary - D Loss Real: 0.7397, D Loss Fake: 7.1141, G Loss: 0.0008

Epoch 2/20
Step 1987/1987 | D Loss Real: 0.7415, D Loss Fake: 7.2167, G Loss: 0.0007
Epoch 2 Summary - D Loss Real: 0.7397, D Loss Fake: 7.1953, G Loss: 0.0008

Epoch 3/20
Step 1987/1987 | D Loss Real: 0.7296, D Loss Fake: 7.2365, G Loss: 0.0007
Epoch 3 Summary - D Loss Real: 0.7397, D Loss Fake: 7.2264, G Loss: 0.0007

Epoch 4/20
Step 1987/1987 | D Loss Real: 0.7408, D Loss Fake: 7.2453, G Loss: 0.0007
Epoch 4 Summary - D Loss Real: 0.7397, D Loss Fake: 7.2415, G Loss: 0.0007

Epoch 5/20
Step 1987/1987 | D Loss Real: 0.7323, D Loss Fake: 7.2481, G Loss: 0.0007
Epoch 5 Summary - D Loss Real: 0.7397, D Loss Fake: 7.2469, G Loss: 0.0007

Epoch 6/20
Step 1987/1987 | D Loss Real: 0.7302, D Loss Fake: 7.2502, G Loss: 0.0007
Epoch 6 Summary - D Loss Real: 0.7397, D Loss Fake: 7.2488, G Loss: 0.0007

Epoch 7/20
Step 1987/1987 | D Los

: 

In [ ]:
plt.plot(d_loss_real, label='Discriminator Real Loss')
plt.plot(d_loss_fake, label='Discriminator Fake Loss')
plt.plot(g_loss, label='Generator Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
def generate_faces(generator, latent_dim, n_samples=5):
    noise = np.random.normal(0, 1, (n_samples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2  # Rescale to [0, 1]
    return generated_images

In [ ]:
generated_images = generate_faces(generator, latent_dim)
plt.figure(figsize=(10, 5))
for i in range(len(generated_images)):
    plt.subplot(1, len(generated_images), i+1)
    plt.imshow(generated_images[i])
    plt.axis('off')
plt.show()